## Evaluator-Optimizer Workflow
In this workflow, one LLM call generates a response while another provides evaluation and feedback in a loop.

### When to use this workflow
This workflow is particularly effective when we have:

- Clear evaluation criteria
- Value from iterative refinement

The two signs of good fit are:

- LLM responses can be demonstrably improved when feedback is provided
- The LLM can provide meaningful feedback itself

In [ ]:
from util import llm_call, extract_xml

def generate(prompt: str, task: str, context: str = "") -> tuple[str, str]:
    """Generate and improve a solution based on feedback."""
    full_prompt = f"{prompt}\n{context}\nTask: {task}" if context else f"{prompt}\nTask: {task}"
    response = llm_call(full_prompt)
    thoughts = extract_xml(response, "thoughts")
    result = extract_xml(response, "response")
    
    print("\n=== GENERATION START ===")
    print(f"Thoughts:\n{thoughts}\n")
    print(f"Generated:\n{result}")
    print("=== GENERATION END ===\n")
    
    return thoughts, result

def evaluate(prompt: str, content: str, task: str) -> tuple[str, str]:
    """Evaluate if a solution meets requirements."""
    full_prompt = f"{prompt}\nOriginal task: {task}\nContent to evaluate: {content}"
    response = llm_call(full_prompt)
    evaluation = extract_xml(response, "evaluation")
    feedback = extract_xml(response, "feedback")
    
    print("=== EVALUATION START ===")
    print(f"Status: {evaluation}")
    print(f"Feedback: {feedback}")
    print("=== EVALUATION END ===\n")
    
    return evaluation, feedback

def loop(task: str, evaluator_prompt: str, generator_prompt: str) -> tuple[str, list[dict]]:
    """Keep generating and evaluating until requirements are met."""
    memory = []
    chain_of_thought = []
    
    thoughts, result = generate(generator_prompt, task)
    memory.append(result)
    chain_of_thought.append({"thoughts": thoughts, "result": result})
    
    while True:
        evaluation, feedback = evaluate(evaluator_prompt, result, task)
        if evaluation == "PASS":
            return result, chain_of_thought
            
        context = "\n".join([
            "Previous attempts:",
            *[f"- {m}" for m in memory],
            f"\nFeedback: {feedback}"
        ])
        
        thoughts, result = generate(generator_prompt, task, context)
        memory.append(result)
        chain_of_thought.append({"thoughts": thoughts, "result": result})

### Example Use Case: Iterative coding loop



In [2]:
evaluator_prompt = """
Evaluate this following code implementation for:
1. code correctness
2. time complexity
3. style and best practices

You should be evaluating only and not attemping to solve the task.
Only output "PASS" if all criteria are met and you have no further suggestions for improvements.
Output your evaluation concisely in the following format.

<evaluation>PASS, NEEDS_IMPROVEMENT, or FAIL</evaluation>
<feedback>
What needs improvement and why.
</feedback>
"""

generator_prompt = """
Your goal is to complete the task based on <user input>. If there are feedback 
from your previous generations, you should reflect on them to improve your solution

Output your answer concisely in the following format: 

<thoughts>
[Your understanding of the task and feedback and how you plan to improve]
</thoughts>

<response>
[Your code implementation here]
</response>
"""

task = """
<user input>
Implement a Stack with:
1. push(x)
2. pop()
3. getMin()
All operations should be O(1).
</user input>
"""

loop(task, evaluator_prompt, generator_prompt)



=== GENERATION START ===
Thoughts:

To implement a stack with O(1) operations including getMin(), I'll use two stacks:
1. A main stack to store actual elements
2. A min stack to track minimum elements at each step

Key considerations:
- When pushing, update min stack if new element is smaller or equal
- When popping, remove from min stack if it matches top of main stack
- This ensures O(1) time complexity for all operations
- Space complexity will be O(n)


Generated:

class MinStack:
    def __init__(self):
        self.stack = []
        self.min_stack = []
    
    def push(self, x: int) -> None:
        self.stack.append(x)
        # Push to min stack if empty or x <= current min
        if not self.min_stack or x <= self.min_stack[-1]:
            self.min_stack.append(x)
    
    def pop(self) -> None:
        if not self.stack:
            return
        
        # If popped element is current min, remove from min stack
        if self.stack[-1] == self.min_stack[-1]:
         

('\nclass MinStack:\n    def __init__(self):\n        self.stack = []\n        self.min_stack = []\n    \n    def push(self, x: int) -> None:\n        self.stack.append(x)\n        # Push to min stack if empty or x <= current min\n        if not self.min_stack or x <= self.min_stack[-1]:\n            self.min_stack.append(x)\n    \n    def pop(self) -> None:\n        if not self.stack:\n            return\n        \n        # If popped element is current min, remove from min stack\n        if self.stack[-1] == self.min_stack[-1]:\n            self.min_stack.pop()\n        \n        self.stack.pop()\n    \n    def getMin(self) -> int:\n        if not self.min_stack:\n            return None\n        return self.min_stack[-1]\n',
 [{'thoughts': "\nTo implement a stack with O(1) operations including getMin(), I'll use two stacks:\n1. A main stack to store actual elements\n2. A min stack to track minimum elements at each step\n\nKey considerations:\n- When pushing, update min stack if new 